# **Whale Inference Module**


---
Welcome to our Whale Inference Module!

Given a directory of aerial whale images, this module will detect each whale in each image and predict its lenght axis, width measurements, and polygon mask.

NOTE: If any of the steps fail or the Google Colab Notebook times out, please delete the session on Google Colab by going to ``Runtime -> Disconnect and delete runtime`` and start again.

## **Preliminary Setup**


---


In this section you will install PyTorch, mount/link your Google Drive, specify a base Google Drive directory location, and load the depedencies to run the model.

### **Step 1** : Mount your Google Drive

Do only ONE of the options below

**Option 1:**

> On the left side bar click on the folder icon to open the *Files* dialog. \
Next, click on the *Mount Drive* icon on the top of this dialog (i.e. the 3rd icon that looks like a folder with the Google Drive logo) \
You will see a popup in the lower left corner saying "Mounting Google Drive...". Once your Google Drive is mounted you should see all of the contents of your drive under this *Files* dialog


**Option 2:**

> Or you can run the code cell below, by clicking on the *Play* button on the cell. \
This will create and link that you have to navigate to, in which you will have to copy an authentication token and paste in the input field that appears under the code
cell. \
If successful it will print out a succesful mounting of your drive message.   


In [ ]:
#@title
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'


Mounted at /content/mnt


### **Step 2** : Specify Directory of Technical Components

In the form cell below, type in or paste in the path to the specific Google Drive folder you uploaded our provided depedency package and model code.

Initially, you should see a placeholder name for the variable ` base_drive_dir `, which is meant to visualize what your input should look like. Pay close attention to end with “/”.

The underlying code for the form cell will automatically run if you insert something new in the field(s). If you do not change out the value in the field you should click the *Play* button on the cell.




In [ ]:
#@title Base Google Drive Directory { run: "auto" }
#@markdown Please specify Google Drive directory that contains the dependecy package and model code provided for the field `base_drive_dir`
import os
base_drive_dir = "/content/mnt/MyDrive" #@param {type:"string"}

if not os.path.isdir(base_drive_dir):
    print("Base folder does not exist. Please re-enter path.")

Base folder does not exist. Please re-enter path.


### **Step 3** : Loading Dependencies

In this step, we load the dependencies by simply clicking the *Play* button on the code cell below. This step may take some time.

In [ ]:
#@title
os.chdir(base_drive_dir)

if not os.path.exists(os.path.join(base_drive_dir, 'detectron2')):
  print("Installing Detectron2:")
  !git clone https://github.com/facebookresearch/detectron2.git
  !python -m pip install -e detectron2
  print("Detectron2 Installed")
else:
  print("Detectron2 is already present. Installing Detectron2:")
  !python -m pip install -e detectron2
  print("Detectron2 Installed")

os.chdir(os.path.join(base_drive_dir,'detectron2'))

# Common imports (already available in Colab)
import json
import random
import numpy as np
import pandas as pd
import cv2
import glob
import math
from collections import defaultdict
import datetime
from shapely.geometry import LineString
from math import hypot
import warnings
from tqdm.notebook import tqdm
import copy
import random
random.seed(42)

# Detectron2 imports (not available in Colab)
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer


## **Running Inference**


---

In this section you will first specify the input image directory you want to perform inference on and the output directory of where you want to see your results on. And then you will run the model to get an output.

### **Step 1** : Specify Image Directory & Output Directory

In the form cell below, type in or paste in the path to the specific Google Drive folder for your input folder and the output folder.

The `input_dir` field is where you should insert the directory that contains your whale images.
The `output_dir` field is where you should insert the directory that you want your predictions for your images to be written in.

Initially, you should see a placeholder name for both the variables `input_dir` and `output_dir`, which is meant to visualize what your input should look like. Pay close attention to end with “/”.

The underlying code for the form cell will automatically run if you insert something new in the field(s). If you do not change out the value in the field you should click the *Play* button on the cell.

In [ ]:
#@title Input & Output Directories { run: "auto" }
#@markdown Please insert the image directory path for `input_dir` and the directory you want your outputs in for `output_dir`

input_dir = "/content/mnt/MyDrive/" #@param {type:"string"}
output_dir = "/content/mnt/MyDrive/" #@param {type:"string"}

if not os.path.isdir(input_dir):
    print("Input folder does not exist. Please re-enter path.")

# if not os.path.isdir(output_dir):
#     print("Output folder does not exist. Please re-enter path.")


### **Step 2** : Configure Model

In this step, we make initial configurations to run inference by simply clicking the *Play* buttons next to the code cells below. Make sure to click each button in order and before clicking the next one make sure that the code cell has stopped running (i.e. the loading wheel on the *Play* will have stopped).


In [ ]:
#@title

## Defining function for timeouts
# import errno
# import os
# import signal
# import functools

# class TimeoutError(Exception):
#     pass

# def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
#     def decorator(func):
#         def _handle_timeout(signum, frame):
#             raise TimeoutError(error_message)

#         @functools.wraps(func)
#         def wrapper(*args, **kwargs):
#             signal.signal(signal.SIGALRM, _handle_timeout)
#             signal.alarm(seconds)
#             try:
#                 result = func(*args, **kwargs)
#             finally:
#                 signal.alarm(0)
#             return result

#         return wrapper

#     return decorator


os.chdir(os.path.join(base_drive_dir, 'Detectron2_Models.zip (Unzipped Files)'))
# os.chdir(base_drive_dir + 'Detectron2 Package')

print("* Configuring model...")

## Setting up the configs for the keypoint detection model
cfg1 = get_cfg()
cfg1.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg1.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
cfg1.OUTPUT_DIR = os.path.join(base_drive_dir, 'Detectron2_Models.zip (Unzipped Files)/july_2023_keypoint_model/keypoint_detection_final_lr_0.00225_batch_size_2_epochs_2000_sbatch')

## Setting up the configs for the segmentation model
cfg2 = get_cfg()
cfg2.MODEL.MASK_ON = True
cfg2.MODEL.KEYPOINT_ON = True
cfg2.OUTPUT_DIR = os.path.join(base_drive_dir, 'Detectron2_Models.zip (Unzipped Files)/july_2023_segmentation_model/segmentation_masks_final_lr_0.00225_batch_size_2_epochs_2500_sbatch')
cfg2.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg2.SOLVER.IMS_PER_BATCH = 2
cfg2.SOLVER.BASE_LR = 0.00025
cfg2.SOLVER.MAX_ITER = 2500
cfg2.SOLVER.STEPS = []        # do not decay learning rate
cfg2.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg2.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (whale)
cfg2.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5


## Loading up the model weights for the keypoint model
cfg1.MODEL.WEIGHTS = os.path.join(cfg1.OUTPUT_DIR, "model_final.pth")
cfg1.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
predictor1 = DefaultPredictor(cfg1)
print("* Done configuring Predictor 1")

## Loading up the model weights for the segmentation model
cfg2.MODEL.WEIGHTS = os.path.join(cfg2.OUTPUT_DIR, "model_final.pth")
cfg2.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor2 = DefaultPredictor(cfg2)
print("* Done configuring Predictor 2")

## Find intersection between the segmentation mask and the 5% width lines
## These points are the endpoints of the 5% width lines
# @timeout(150)
def findIntersections(mask,slope_calc_x1,slope_calc_y1,slope_calc_x2,slope_calc_y2,xvalues,yvalues):
    ## Find all points on the 5% width lines where
    ## [aX,aY] and [bX,bY] are the endpoints to calculate the slope
    ## Line is drawn through [pX,pY]
    def getAllPointsOnLine(aX, aY, bX, bY,pX,pY, length,mask):
        vX = bX-aX
        vY = bY-aY

        if(vX == 0 or vY == 0):
            return 0, 0, 0, 0
        mag = math.sqrt(vX*vX + vY*vY)
        vX = vX / mag
        vY = vY / mag
        temp = vX
        vX = 0-vY
        vY = temp
        cX=[]
        cY=[]
        dX=[]
        dY=[]
        for i in range(length):
            cX_temp=int(pX + vX * i)
            cY_temp=int(pY + vY * i)
            if cX_temp<mask.shape[1] and cY_temp<mask.shape[0] and cX_temp>=0 and cY_temp>=0:
                cX.append(cX_temp)
                cY.append(cY_temp)

            dX_temp=int(pX - vX * i)
            dY_temp=int(pY - vY * i)
            if dX_temp<mask.shape[1] and dY_temp<mask.shape[0] and dX_temp>=0 and dY_temp>=0:
                dX.append(dX_temp)
                dY.append(dY_temp)

        return (cX), (cY), (dX), (dY)

    cXAll=[]
    cYAll=[]
    dXAll=[]
    dYAll=[]

    intersection=defaultdict(list)
    for i in range(1,len(xvalues)-1):
        cX,cY,dX,dY=getAllPointsOnLine(slope_calc_x1[i],slope_calc_y1[i],slope_calc_x2[i],slope_calc_y2[i],xvalues[i],yvalues[i],400,mask)

        for j in range(1,len(cX)-1):

            if int(mask[cY[j]][cX[j]])!=int(mask[cY[j+1]][cX[j+1]]):
                intersection[i*5].append([cX[j],cY[j]])
            if int(mask[dY[j]][dX[j]])!=int(mask[dY[j+1]][dX[j+1]]):
                intersection[i*5].append([dX[j],dY[j]])
        if (i)*5 not in intersection.keys():
            intersection[i*5]=[]

    return intersection



## Find coordinates of the perpendicular line through [pX,pY]
## [aX,aY] and [bX,bY] are points on either side of [pX,pY] on the axis
# @timeout(150)
def getPerpCoord(aX, aY, bX, bY,pX,pY, length):
    vX = bX-aX
    vY = bY-aY

    if(vX == 0 or vY == 0):
        return 0, 0, 0, 0
    mag = math.sqrt(vX*vX + vY*vY)
    vX = vX / mag
    vY = vY / mag
    temp = vX
    vX = 0-vY
    vY = temp
    cX = pX + vX * length
    cY = pY + vY * length
    dX = pX - vX * length
    dY = pY - vY * length
    return int(cX), int(cY), int(dX), int(dY)   # end getPerpCoord()



## Find all points on the 5% width lines where
## [aX,aY] and [bX,bY] are the endpoints to calculate the slope
## Line is drawn through [pX,pY]
# @timeout(150)
def getAllPointsOnLine(aX, aY, bX, bY,pX,pY, length):
    vX = bX-aX
    vY = bY-aY

    if(vX == 0 or vY == 0):
        return 0, 0, 0, 0
    mag = math.sqrt(vX*vX + vY*vY)
    vX = vX / mag
    vY = vY / mag
    temp = vX
    vX = 0-vY
    vY = temp
    cX=[]
    cY=[]
    dX=[]
    dY=[]
    for i in range(length):
        cX.append(int(pX + vX * length))
        cY.append(int(pY + vY * length))
        dX.append(int(pX - vX * length))
        dY.append(int(pY - vY * length))
    return (cX), (cY), (dX), (dY)   # end getAllPointsOnLine()


## Function to calculate the slope of the axis at 5% intervals
# @timeout(150)
def calculateSlopes(im, intersection_points, z):
    xvalues = intersection_points[:, 0]
    yvalues = intersection_points[:, 1]
    slope_calc_x1=[]
    slope_calc_x2=[]

    for i in range(len(xvalues)):
        slope_calc_x1.append(xvalues[i]-2)
        slope_calc_x2.append(xvalues[i]+2)

    slope_calc_y1=np.polyval(z,slope_calc_x1)
    slope_calc_y2=np.polyval(z,slope_calc_x2)

    slopes=[]

    for i in range(len(slope_calc_x1)):
        slopes.append((slope_calc_y2[i]-slope_calc_y1[i])/(slope_calc_y2[i]-slope_calc_y1[i]))

    return slopes,slope_calc_x1,slope_calc_y1,slope_calc_x2,slope_calc_y2,xvalues,yvalues   # end calculateSlopes()


## Draw perpendicular lines at every 5% of the axis
# @timeout(150)
def drawPerps(im,slopes,slope_calc_x1,slopes_calc_y1,slopes_calc_x2,slopes_calc_y2,xvalues,yvalues):
    perpLines=[]
    for i in range(len(slopes)):
        perpLines.append(getPerpCoord(slope_calc_x1[i],slope_calc_y1[i],slope_calc_x2[i],slope_calc_y2[i],xvalues[i],yvalues[i],400))

    for i in range(len(perpLines)):
        if i==len(perpLines)-4:
            im = cv2.line(im, [perpLines[i][0],perpLines[i][1]], [perpLines[i][2],perpLines[i][3]], (0,0,255), 2)
        else:
            im = cv2.line(im, [perpLines[i][0],perpLines[i][1]], [perpLines[i][2],perpLines[i][3]], (0,255,255), 2)

    return im   # end drawPerps()


## Function to print the Whale ID on the top right corner of the image
# @timeout(150)
def markWhaleID(im,instance_no,index,instances):
    font = cv2.FONT_HERSHEY_DUPLEX
    pred_boxes= outputs2["instances"].pred_boxes.tensor.cpu().numpy()
    org=(int(pred_boxes[index][0])+5,int(pred_boxes[index][1])+100)
    fontScale = 2
    color = (0, 238, 238)
    font_thickness = 2
    im = cv2.putText(im, 'Whale '+str(instance_no+1), org, font, fontScale, color, font_thickness, cv2.LINE_AA)

    start_point = (int(pred_boxes[index][0]),int(pred_boxes[index][1]))
    end_point = (int(pred_boxes[index][2]),int(pred_boxes[index][3]))
    rectangle_thickness = 4
    im = cv2.rectangle(im, start_point, end_point, color, rectangle_thickness)


    return im   # end markWhaleID()


## Function to calculate Euclidean distance
# @timeout(150)
def calculateWhaleLength(x1, y1, x2, y2):
    return np.sqrt((x2-x1)**2 + (y2-y1)**2)

## nseg = Number of segments the polynomial curve needs to be divided into
## z = polynomial curve
## x = X-coordinate of all keypoints
## y = Y-coordinate of all keypoints
# @timeout(150)
def divide_curve_into_n_segments(nseg, z, x, y):
    total_length = arclength(f, z, min(x), max(x))
    parts = 1001
    # nseg-=2
    seg_length = total_length/parts
    start_x = min(x)
    start_y = y[np.argmin(x)]
    theta = np.linspace(0, 2*np.pi, 100)

    r = seg_length
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    x_along_whale_axis = [x for x in range(min(x), max(x))]
    whale_axis = np.polyval(z, x_along_whale_axis)

    all_intersection_points = np.zeros((parts,2))

    for i in range(parts):
        new_x1 = x1 + start_x
        new_x2 = x2 + start_y

        first_line = LineString(np.column_stack((new_x1, new_x2)))
        second_line = LineString(np.column_stack((x_along_whale_axis, whale_axis)))
        intersection = first_line.intersection(second_line)
        # print("Intersection geom:", intersection.geom_type)
        if intersection.geom_type == 'MultiPoint':
            # intersection_points = np.asarray(intersection)

            # for point in intersection_points:
            #     if point[0] > start_x:
            #         point_of_intersection = point

            for point in intersection.geoms:
                if point.x > start_x:
                    point_of_intersection = np.asarray([point.x, point.y])
        elif intersection.geom_type == 'Point':
            # point_of_intersection = np.asarray(intersection)
            point_of_intersection = np.asarray([intersection.x, intersection.y])
        start_x = point_of_intersection[0]
        start_y = point_of_intersection[1]
        # print(type(point_of_intersection))
        # point_of_intersection = [point_of_intersection.x, point_of_intersection.y]
        all_intersection_points[i,:] = point_of_intersection

    intersection_values_at_seg = np.zeros((nseg+1,2))

    j=0
    for i in range(0, parts, parts//nseg):
        intersection_values_at_seg[j] = all_intersection_points[i,:]
        j+=1
#     plt.plot(x_along_whale_axis, whale_axis)
#     plt.plot(intersection_values_at_seg[:,0],intersection_values_at_seg[:,1], markersize = 5, marker="o")
#     plt.show()
    if x[0]<x[-1]:
        return intersection_values_at_seg
    else:
        return np.flip(intersection_values_at_seg, axis = 0)

# @timeout(150)
def f(z,x):
    return np.polyval(z,x)

# @timeout(150)
def arclength(f, z, a, b, tol=1e-6):
    """Compute the arc length of function f(x) for a <= x <= b. Stop
    when two consecutive approximations are closer than the value of
    tol.
    """
    nsteps = 1  # number of steps to compute
    oldlength = 1.0e20
    length = 1.0e10
    while abs(oldlength - length) >= tol:
        nsteps *= 2
        fx1 = f(z,a)
        xdel = (b - a) / nsteps  # space between x-values
        oldlength = length
        length = 0
        for i in range(1, nsteps + 1):
            fx0 = fx1  # previous function value
            fx1 = f(z, a + i * (b - a) / nsteps)  # new function value
            length += hypot(xdel, fx1 - fx0)  # length of small line segment
    return length

# @timeout(150)
def find_mappings(keypoint_output, segmentation_output):
    number_of_keypoint_instances = len(keypoint_output["instances"].pred_boxes)
    number_of_segmentation_instances = len(segmentation_output["instances"].pred_boxes)

    keypoint_pred_boxes = keypoint_output["instances"].pred_boxes.tensor.cpu().numpy()
    segmentation_pred_boxes = segmentation_output["instances"].pred_boxes.tensor.cpu().numpy()
    mapping = []
    already_mapped_keypoints = set()
    already_mapped_segmentation = set()
    diff_dict = {}

    # for k in range(len(keypoint_output["instances"].pred_boxes)):
    for k in range(number_of_keypoint_instances):
        diff_arr = np.zeros(number_of_segmentation_instances)
        # for i in range(len(segmentation_output["instances"].pred_boxes)):
        for i in range(number_of_segmentation_instances):
            difference = 0
            for j in range(4):
                difference = difference+abs(segmentation_pred_boxes[i, j]-keypoint_pred_boxes[k, j])
            diff_arr[i] = (difference)
            diff_dict[difference] = [k, i] #[Keypoint, Segmentation]

    all_differences = list(diff_dict.keys())
    all_differences.sort()

    number_of_instances_required = min(number_of_segmentation_instances, number_of_keypoint_instances)
    final_mapping = []

    i = 0

    while len(final_mapping)!= number_of_instances_required:
        diff_check = all_differences[i]
        mapping = diff_dict[diff_check]
        if mapping[0] not in already_mapped_keypoints and mapping[1] not in already_mapped_segmentation:
            final_mapping.append(mapping)
            already_mapped_keypoints.add(mapping[0])
            already_mapped_segmentation.add(mapping[1])
        i+=1

    return final_mapping

def overlay_mask_on_image(img, mask, alpha=0.5):
  colors = [[0,255,0], #'green'
            [204,0,102], #'magenta',
            [204, 102, 0], #'gold',
            [0,0,255], #'blue',
            [0,0,0], #'black',
            [255, 0, 0]] #'red'
  color = random.choice(colors)
  r = mask*color[0]
  g = mask*color[1]
  b = mask*color[2]

  contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  bgr_mask = np.asarray([b,g,r]).swapaxes(0,2).swapaxes(0,1)
  img = np.asarray(img, np.float32)
  masked_img = cv2.addWeighted(bgr_mask, alpha, img, 1-alpha, 0, img)
  masked_img = cv2.drawContours(masked_img, contours, -1, color, 2)

  return masked_img


### **Step 3** : Obtain your Outputs

In this step, we run inference and obtain results by simply clicking the *Play* button next to the code cell below.

As the code cell runs, you will see feedback printed out under the cell as the model runs on each subsequent image, and once complete it will give a confirmation that describes where you can view your outputs at.

If you want to view the outputs in this notebook, set ``viz = True`` in the code.

In [ ]:
#@title

viz = False

import traceback
import signal
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

def handler(signum, frame):
  # print("Forever is over!")
  raise Exception("Timed out")

## Create required output folders
ct = datetime.datetime.now()
date_str = ct.strftime("%Y_%d_%m_%H%M%S")
output_dir = os.path.join(output_dir, 'output_'+ date_str)
os.makedirs(output_dir)
os.makedirs(os.path.join(output_dir, 'predicted_images'))
os.makedirs(os.path.join(output_dir, 'masks'))
print("* Begun prediction...")

final_intersection=[]
number_of_files=len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))])
count=1
nseg = 20

# if input_dir[-1] == '/':
#     input_dir_select_all = input_dir+'*'
# else:
#     input_dir_select_all=input_dir+'/*'

input_dir_select_all = os.path.join(input_dir, '*')

all_files = glob.glob(input_dir_select_all)

final_intersection=[]
number_of_files=len([name for name in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, name))])
count=1
debug = True
# if input_dir[-1]=='/':
#     input_dir_select_all=input_dir+'*'
# else:
#     input_dir_select_all=input_dir+'/*'

for filename in tqdm(glob.glob(os.path.join(input_dir, '*'))):
    # print ("Processing image ",count,'/',number_of_files)
    count+=1
    # if count==10:
    #   break
    # if count<637:
    #   continue
    # print(filename)
    try:
      im = cv2.imread(filename)

      outputs1 = predictor1(im)
      outputs2 = predictor2(im)
      im_temp=copy.deepcopy(im)

      mapping_segmentation_to_keypoint = find_mappings(outputs1, outputs2)
      v = False
    except Exception:
        if debug:
          print(filename)
          traceback.print_exc()
      # print(len(mapping_segmentation_to_keypoint))
    for i in range(len(mapping_segmentation_to_keypoint)):
      im = copy.deepcopy(im_temp)
      signal.signal(signal.SIGALRM, handler)
      signal.alarm(60)
      try:
          instance_no = mapping_segmentation_to_keypoint[i][0] #Keypoint instance
          index = mapping_segmentation_to_keypoint[i][1] #Segmentation instance
          diff_arr=[]
          tens=outputs1["instances"].pred_keypoints[instance_no]
          kp_df=tens.cpu().numpy()

          midpoint = kp_df[:,:2].astype(int)

          x,y=zip(*midpoint)
          lspace=np.linspace(min(x),max(x),100)
          z=np.polyfit(x,y,5)
          draw_x=lspace
          draw_y=np.polyval(z,draw_x)
          draw_points = (np.asarray([draw_x, draw_y]).T).astype(np.int32)
          im=cv2.polylines(im, [draw_points], False, (255,255,255),thickness=2)
          intersection_points = divide_curve_into_n_segments(nseg, z, x, y)
          slopes,slope_calc_x1,slope_calc_y1,slope_calc_x2,slope_calc_y2,xvalues,yvalues=calculateSlopes(im,intersection_points,z)

          intersection=findIntersections(outputs2["instances"].pred_masks[index],slope_calc_x1,slope_calc_y1,slope_calc_x2,slope_calc_y2,xvalues,yvalues)
          area_in_sq_px=sum(sum(outputs2["instances"].pred_masks[index])).cpu().numpy()
          cv2.imwrite(os.path.join(output_dir,'masks/'+".".join((filename.split("/")[-1]).split('.')[:-1])+'.Whale.'+str(instance_no+1)+'.jpg'),outputs2["instances"].pred_masks[index].cpu().numpy().astype('float32')*255)

          im = markWhaleID(im,instance_no,index,outputs2["instances"])
          whaleLength = calculateWhaleLength(int(tens[0][0]),int(tens[0][1]),int(tens[4][0]),int(tens[4][1]))
          intersection_val = [".".join((filename.split("/")[-1]).split('.')[:-1]),instance_no+1,int(tens[0][0]),int(tens[0][1]),int(tens[4][0]),int(tens[4][1]),whaleLength,area_in_sq_px]
          for key,val in intersection.items():
              if len(val)==2:
                  im = cv2.line(im, tuple(val[0]), tuple(val[1]), thickness=2, color=(0, 255, 0))
                  if (np.polyval(z,val[0][0]) - val[0][1])>0: ## So that all (X1,Y1) are on one side and (X2,Y2) are on the other side
                    intersection_val.append(val[0][0])
                    intersection_val.append(val[0][1])
                    intersection_val.append(val[1][0])
                    intersection_val.append(val[1][1])
                  else:
                    intersection_val.append(val[1][0])
                    intersection_val.append(val[1][1])
                    intersection_val.append(val[0][0])
                    intersection_val.append(val[0][1])
              else:
                  intersection_val.append(-1)
                  intersection_val.append(-1)
                  intersection_val.append(-1)
                  intersection_val.append(-1)
          final_intersection.append(intersection_val)

          v = Visualizer(im[:, :, ::-1],
                  # scale=0.8
                  scale=1
                  )
          try:
            if v:

              # print(output_dir+'predicted_images/'+".".join((filename.split("/")[-1]).split('.')[:-1])+'.WidthMarked.jpg')
              pred_mask = outputs2["instances"].pred_masks[index].cpu().numpy().astype('float32')
              img_with_overlayed_mask = overlay_mask_on_image(im, pred_mask, alpha=0.2)
              cv2.imwrite(os.path.join(output_dir,'predicted_images/'+".".join((filename.split("/")[-1]).split('.')[:-1])+f'Whale.{str(instance_no+1)}.WidthMarked.jpg'),img_with_overlayed_mask)
              if viz:
                out1 = v.draw_instance_predictions(outputs2["instances"].to("cpu"))
                plt.figure(figsize=(12,8))
                plt.imshow(cv2.cvtColor(out1.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
                plt.show()
          # except:
          except Exception as e:
              if debug:
                print(repr(e))
                traceback.print_exc()
              # print("Error while processing ",filename.split("/")[-1])
              # print(sys.exc_info()[0])
              # #raise
      except Exception:
        if debug:
          print(filename)
          traceback.print_exc()
    # try:
    #   if v:
    #     out1 = v.draw_instance_predictions(outputs2["instances"].to("cpu"))
    #     # print(output_dir+'predicted_images/'+".".join((filename.split("/")[-1]).split('.')[:-1])+'.WidthMarked.jpg')
    #     cv2.imwrite(os.path.join(output_dir,'predicted_images/'+".".join((filename.split("/")[-1]).split('.')[:-1])+f'WidthMarked.jpg'),out1.get_image()[:, :, ::-1])
    #     # plt.figure(figsize=(12,8))
    #     # plt.imshow(cv2.cvtColor(out1.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    #     # plt.show()
    # # except:
    # except Exception:
    #     if debug:
    #       print(filename)
    #       traceback.print_exc()
    #     # print("Error while processing ",filename.split("/")[-1])
    #     # print(sys.exc_info()[0])
    #     # #raise


# cols=["Image.ID","Whale.ID","Rostrum.X","Rostrum.Y","Fluke.Middle.X","Fluke.Middle.Y","Total.Length","Area",'Endpoint.Width.5.X1',
#       'Endpoint.Width.5.Y1', 'Endpoint.Width.5.X2', 'Endpoint.Width.5.Y2', 'Endpoint.Width.10.X1','Endpoint.Width.10.Y1',
#       'Endpoint.Width.10.X2', 'Endpoint.Width.10.Y2', 'Endpoint.Width.15.X1','Endpoint.Width.15.Y1', 'Endpoint.Width.15.X2',
#       'Endpoint.Width.15.Y2', 'Endpoint.Width.20.X1','Endpoint.Width.20.Y1', 'Endpoint.Width.20.X2', 'Endpoint.Width.20.Y2',
#       'Endpoint.Width.25.X1','Endpoint.Width.25.Y1', 'Endpoint.Width.25.X2', 'Endpoint.Width.25.Y2', 'Endpoint.Width.30.X1',
#       'Endpoint.Width.30.Y1', 'Endpoint.Width.30.X2', 'Endpoint.Width.30.Y2', 'Endpoint.Width.35.X1','Endpoint.Width.35.Y1',
#       'Endpoint.Width.35.X2', 'Endpoint.Width.35.Y2', 'Endpoint.Width.40.X1','Endpoint.Width.40.Y1', 'Endpoint.Width.40.X2',
#       'Endpoint.Width.40.Y2', 'Endpoint.Width.45.X1','Endpoint.Width.45.Y1', 'Endpoint.Width.45.X2', 'Endpoint.Width.45.Y2',
#       'Endpoint.Width.50.X1','Endpoint.Width.50.Y1', 'Endpoint.Width.50.X2', 'Endpoint.Width.50.Y2', 'Endpoint.Width.55.X1',
#       'Endpoint.Width.55.Y1', 'Endpoint.Width.55.X2', 'Endpoint.Width.55.Y2', 'Endpoint.Width.60.X1','Endpoint.Width.60.Y1',
#       'Endpoint.Width.60.X2', 'Endpoint.Width.60.Y2', 'Endpoint.Width.65.X1','Endpoint.Width.65.Y1', 'Endpoint.Width.65.X2',
#       'Endpoint.Width.65.Y2', 'Endpoint.Width.70.X1','Endpoint.Width.70.Y1', 'Endpoint.Width.70.X2', 'Endpoint.Width.70.Y2',
#       'Endpoint.Width.75.X1','Endpoint.Width.75.Y1', 'Endpoint.Width.75.X2', 'Endpoint.Width.75.Y2', 'Endpoint.Width.80.X1',
#       'Endpoint.Width.80.Y1', 'Endpoint.Width.80.X2', 'Endpoint.Width.80.Y2', 'Endpoint.Width.85.X1','Endpoint.Width.85.Y1',
#       'Endpoint.Width.85.X2', 'Endpoint.Width.85.Y2', 'Endpoint.Width.90.X1','Endpoint.Width.90.Y1', 'Endpoint.Width.90.X2',
#       'Endpoint.Width.90.Y2', 'Endpoint.Width.95.X1','Endpoint.Width.95.Y1', 'Endpoint.Width.95.X2', 'Endpoint.Width.95.Y2']

# intersection_df=pd.DataFrame(final_intersection,columns=cols)

# j=0
# for i in range(8,intersection_df.shape[1],4):
#     j+=5
#     cols=intersection_df.columns[i:i+4]
#     intersection_df["width_"+str(j)]=np.sqrt((intersection_df[cols[0]]-intersection_df[cols[2]])**2+(intersection_df[cols[1]]-intersection_df[cols[3]])**2)
# intersection_df.replace(-1,np.nan,inplace=True)



# intersection_df.to_csv(os.path.join(output_dir,"predicted_data.csv"))

# print("All images have been processed. Outputs can be found in ",output_dir)


# except:
#     print("Error while processing ", filename.split("/")[-1])
#     print(sys.exc_info()[0])
#     #raise


cols = ["Image.ID", "Whale.ID", "Rostrum.X", "Rostrum.Y", "Fluke.Middle.X", "Fluke.Middle.Y", "Total.Length", "Area",
      'Endpoint.Width.5.X1', 'Endpoint.Width.5.Y1', 'Endpoint.Width.5.X2', 'Endpoint.Width.5.Y2',
      'Endpoint.Width.10.X1', 'Endpoint.Width.10.Y1', 'Endpoint.Width.10.X2', 'Endpoint.Width.10.Y2',
      'Endpoint.Width.15.X1', 'Endpoint.Width.15.Y1', 'Endpoint.Width.15.X2', 'Endpoint.Width.15.Y2',
      'Endpoint.Width.20.X1', 'Endpoint.Width.20.Y1', 'Endpoint.Width.20.X2', 'Endpoint.Width.20.Y2',
      'Endpoint.Width.25.X1', 'Endpoint.Width.25.Y1', 'Endpoint.Width.25.X2', 'Endpoint.Width.25.Y2',
      'Endpoint.Width.30.X1', 'Endpoint.Width.30.Y1', 'Endpoint.Width.30.X2', 'Endpoint.Width.30.Y2',
      'Endpoint.Width.35.X1', 'Endpoint.Width.35.Y1', 'Endpoint.Width.35.X2', 'Endpoint.Width.35.Y2',
      'Endpoint.Width.40.X1', 'Endpoint.Width.40.Y1', 'Endpoint.Width.40.X2', 'Endpoint.Width.40.Y2',
      'Endpoint.Width.45.X1', 'Endpoint.Width.45.Y1', 'Endpoint.Width.45.X2', 'Endpoint.Width.45.Y2',
      'Endpoint.Width.50.X1', 'Endpoint.Width.50.Y1', 'Endpoint.Width.50.X2', 'Endpoint.Width.50.Y2',
      'Endpoint.Width.55.X1', 'Endpoint.Width.55.Y1', 'Endpoint.Width.55.X2', 'Endpoint.Width.55.Y2',
      'Endpoint.Width.60.X1', 'Endpoint.Width.60.Y1', 'Endpoint.Width.60.X2', 'Endpoint.Width.60.Y2',
      'Endpoint.Width.65.X1', 'Endpoint.Width.65.Y1', 'Endpoint.Width.65.X2', 'Endpoint.Width.65.Y2',
      'Endpoint.Width.70.X1', 'Endpoint.Width.70.Y1', 'Endpoint.Width.70.X2', 'Endpoint.Width.70.Y2',
      'Endpoint.Width.75.X1', 'Endpoint.Width.75.Y1', 'Endpoint.Width.75.X2', 'Endpoint.Width.75.Y2',
      'Endpoint.Width.80.X1', 'Endpoint.Width.80.Y1', 'Endpoint.Width.80.X2', 'Endpoint.Width.80.Y2',
      'Endpoint.Width.85.X1', 'Endpoint.Width.85.Y1', 'Endpoint.Width.85.X2', 'Endpoint.Width.85.Y2',
      'Endpoint.Width.90.X1', 'Endpoint.Width.90.Y1', 'Endpoint.Width.90.X2', 'Endpoint.Width.90.Y2',
      'Endpoint.Width.95.X1', 'Endpoint.Width.95.Y1', 'Endpoint.Width.95.X2', 'Endpoint.Width.95.Y2']

intersection_df = pd.DataFrame(final_intersection, columns=cols)

j = 0
for i in range(8, intersection_df.shape[1], 4):
    j += 5
    cols = intersection_df.columns[i:i+4]
    intersection_df["width_"+str(j)] = np.sqrt((intersection_df[cols[0]]-intersection_df[cols[2]])**2+(intersection_df[cols[1]]-intersection_df[cols[3]])**2)

intersection_df.replace(-1, np.nan, inplace=True)

nan_array = np.empty(intersection_df.shape[0])
nan_array[:] = np.nan

intersection_df["Peduncle.X"] = nan_array
intersection_df["Peduncle.Y"] = nan_array
intersection_df["Dorsal.Fin.Start.X"] = nan_array
intersection_df["Dorsal.Fin.Start.Y"] = nan_array
intersection_df["Dorsal.Fin.End.X"] = nan_array
intersection_df["Dorsal.Fin.End.Y"] = nan_array
intersection_df["Fluke.Endpoint.X1"] = nan_array
intersection_df["Fluke.Endpoint.Y1"] = nan_array
intersection_df["Fluke.Endpoint.X2"] = nan_array
intersection_df["Fluke.Endpoint.Y2"] = nan_array
intersection_df["Blowhole.X"] = nan_array
intersection_df["Blowole.Y"] = nan_array
intersection_df["Eye.X1"] = nan_array
intersection_df["Eye.Y1"] = nan_array
intersection_df["Eye.X2"] = nan_array
intersection_df["Eye.Y2"] = nan_array

intersection_df.to_csv(os.path.join(output_dir, "predicted_data.csv")) # Write all data to predicted_data.csv

print("* All images have been processed. Outputs can be found in ", output_dir)

## **Completion**


---



To view your outputs, navigate to your output directory in your Google Drive and look for the subfolder name that the code cell above printed out. The subfolder has the naming convention of:  `output_<date>_<time> `
This subfolder will contain a folder of predicted images, a folder of masks, and a CSV file quantifying the predictions.

If your runtime has not timed out you can re-run the module on a different batch of images starting from the beginning of the Running Inference section (and can skip the sections above it as long as the green check marks next to the cells above remains and your base folder has not changed). You can either remove the current batch of images from your `input_dir` and upload the new batch and not change the location. Or simply give a different location path to `input_dir` referring to a new folder with the new images.